In [1]:
import pandas as pd
import torch
import numpy as np
from datetime import datetime
import numpy as np
import datetime
import matplotlib.pyplot as plt
from scipy.stats import normaltest
from sklearn.preprocessing import MinMaxScaler
import random
import copy
from sklearn.kernel_ridge import KernelRidge
import re
import nltk
from nltk import word_tokenize

In [2]:
# previously loaded data from 2013 - 2019 - now running on new data from 2005 to 2019
# data = pd.read_csv("../Current_Data/New_data_updated_names.csv", usecols=['Country','Date','Cases','Deaths',' 
#                   Summary'], parse_dates=["Date"])
data = pd.read_csv("../Current_Data/data2005to2019.csv", usecols=['Country','Date','Cases','Deaths','Summary'],  
                  parse_dates=["Date"])

data.head()


,Country,Date,Cases,Deaths,Summary
0,Benin,2005-01-03,13,8.0,Note that year 2004 had 53 weeks. Hence the fi...
1,Burkina Faso,2005-01-03,106,23.0,Note that year 2004 had 53 weeks. Hence the fi...
2,Ivory Coast,2005-01-03,5,0.0,Note that year 2004 had 53 weeks. Hence the fi...
3,Mali,2005-01-03,18,0.0,Note that year 2004 had 53 weeks. Hence the fi...
4,Niger,2005-01-03,29,4.0,Note that year 2004 had 53 weeks. Hence the fi...


In [3]:
#note chad is at the end of the list because, despite the list being alphabetically organized, in WHO reports
#chad was recorded as Tchad
country_list = ['Benin', 'Burkina Faso', 'Burundi', 'Cameroun', 'Centrafrique', "Ivory Coast", 'Ethiopia', 'Ghana',
               'Guinea', 'Guinea Bissau', 'Gambia', 'Kenya', 'Mali', 'Mauritania', 'Niger', 'Nigeria', 'Dem. Rep. Congo', 
                'Senegal' , 'South Sudan', 'Sudan', 'Sierra Lone', 'Tanzania', 'Chad', 'Togo', 'Uganda']
bordering_countries = {"Benin": ["Togo", "Nigeria", "Niger", "Burkina Faso"], 
                       "Burkina Faso": ["Mali", "Niger", "Benin" ,"Togo", "Ghana", "Ivory Coast"], 
                       "Burundi": ["Dem. Rep. Congo", "Tanzania"], 
                       "Cameroun": ["Centrafrique", "Nigeria", "Guinea", "Chad"], 
                       "Centrafrique": ["Cameroun", "Sudan", "South Sudan","Dem. Rep. Congo", "Chad"], 
                       "Ivory Coast": ["Ghana", "Mali", "Burkina Faso"], 
                       "Ethiopia": ["Sudan", "South Sudan", "Kenya"], 
                       "Ghana":["Guinea", "Ivory Coast", "Burkina Faso", "Togo"], 
                       "Guinea":["Ivory Coast", "Guinea Bissau", "Mali", "Senegal", "Sierra Lone"], 
                       "Guinea Bissau": ["Senegal", "Guinea"], 
                       "Gambia":["Senegal"], 
                       "Kenya": ["Ethiopia", "South Sudan", "Uganda"],
                       "Mali": ["Niger", "Burkina Faso", "Ivory Coast", "Guinea", "Mauritania", "Senegal"], 
                       "Mauritania":["Mali", "Senegal"], 
                       "Niger": ["Benin", "Nigeria", "Burkina Faso", "Mali", "Chad"], 
                       "Nigeria": ["Benin", "Cameroun", "Niger", "Chad"], 
                       "Dem. Rep. Congo": ["Burundi", "Centrafrique", "South Sudan", "Tanzania", "Uganda"], 
                       "Senegal":["Gambia", "Guinea", "Guinea Bissau", "Mali","Mauritania"], 
                       "South Sudan":["Sudan", "Ethiopia", "Kenya", "Dem. Rep. Congo", "Uganda","Centrafrique"], 
                       "Sudan":["Ethiopia", "South Sudan", "Centrafrique", "Chad"], 
                       "Sierra Lone": ["Guinea"], 
                       "Tanzania":["Kenya", "Uganda", "Burundi", "Dem. Rep. Congo"], 
                       "Chad":["Sudan", "Centrafrique", "Cameroun", "Niger", "Nigeria"], 
                       "Togo":["Ghana", "Burkina Faso","Benin", "Guinea"],
                       "Uganda":["Kenya", "South Sudan", "Dem. Rep. Congo", "Tanzania"]}
#this lambda function returns the index of the country from the corresponding country_list array
get_indices = lambda x: country_list.index(str(x).rstrip())
country_list.index("Guinea")

8

In [4]:
#creates a list of bordering countries for each datapoint with the countries represented as indices of country_list array
data_bordering_countries = pd.DataFrame(data["Country"].apply(lambda x: 
                                                              list(
                                                                  map(get_indices, 
                                                                      bordering_countries[str(x).rstrip()]))
                                                                   ))
data.insert(5, "Bordering Countries", data_bordering_countries)
#replace country names in country column with corresponding indices from country_list array
data["Country"] = data["Country"].apply(lambda x: country_list.index(str(x).rstrip()))
data.loc[0:20]

,Country,Date,Cases,Deaths,Summary,Bordering Countries
0,0,2005-01-03,13,8.0,Note that year 2004 had 53 weeks. Hence the fi...,"[23, 15, 14, 1]"
1,1,2005-01-03,106,23.0,Note that year 2004 had 53 weeks. Hence the fi...,"[12, 14, 0, 23, 7, 5]"
2,5,2005-01-03,5,0.0,Note that year 2004 had 53 weeks. Hence the fi...,"[7, 12, 1]"
3,12,2005-01-03,18,0.0,Note that year 2004 had 53 weeks. Hence the fi...,"[14, 1, 5, 8, 13, 17]"
4,14,2005-01-03,29,4.0,Note that year 2004 had 53 weeks. Hence the fi...,"[0, 15, 1, 12, 22]"
5,16,2005-01-03,61,12.0,Note that year 2004 had 53 weeks. Hence the fi...,"[2, 4, 18, 21, 24]"
6,22,2005-01-03,4,0.0,Note that year 2004 had 53 weeks. Hence the fi...,"[19, 4, 3, 14, 15]"
7,23,2005-01-03,7,1.0,Note that year 2004 had 53 weeks. Hence the fi...,"[7, 1, 0, 8]"
8,0,2005-01-10,7,1.0,"At week 2, 2005, the district of Saponé in Bur...","[23, 15, 14, 1]"
9,1,2005-01-10,156,37.0,"At week 2, 2005, the district of Saponé in Bur...","[12, 14, 0, 23, 7, 5]"


In [5]:
def find_next_batch(count, data=data):
    initialCount = count
    found = False
    while not found:
        count += 1
        print(count)
        if count == len(data) - 1 or data["Date"].loc[count + 1] != data["Date"].loc[count]:
            found = True
            break
    return data.iloc[initialCount:count+1]

In [6]:
find_next_batch(0)

1
2
3
4
5
6
7


,Country,Date,Cases,Deaths,Summary,Bordering Countries
0,0,2005-01-03,13,8.0,Note that year 2004 had 53 weeks. Hence the fi...,"[23, 15, 14, 1]"
1,1,2005-01-03,106,23.0,Note that year 2004 had 53 weeks. Hence the fi...,"[12, 14, 0, 23, 7, 5]"
2,5,2005-01-03,5,0.0,Note that year 2004 had 53 weeks. Hence the fi...,"[7, 12, 1]"
3,12,2005-01-03,18,0.0,Note that year 2004 had 53 weeks. Hence the fi...,"[14, 1, 5, 8, 13, 17]"
4,14,2005-01-03,29,4.0,Note that year 2004 had 53 weeks. Hence the fi...,"[0, 15, 1, 12, 22]"
5,16,2005-01-03,61,12.0,Note that year 2004 had 53 weeks. Hence the fi...,"[2, 4, 18, 21, 24]"
6,22,2005-01-03,4,0.0,Note that year 2004 had 53 weeks. Hence the fi...,"[19, 4, 3, 14, 15]"
7,23,2005-01-03,7,1.0,Note that year 2004 had 53 weeks. Hence the fi...,"[7, 1, 0, 8]"


In [7]:
find_next_batch(5057)

5058
5059
5060
5061
5062
5063
5064
5065
5066
5067
5068
5069
5070
5071
5072
5073


,Country,Date,Cases,Deaths,Summary,Bordering Countries
5057,0,2019-12-16,55,4.0,"From week 49 to 52 (December 2019), 8 district...","[23, 15, 14, 1]"
5058,1,2019-12-16,87,1.0,"From week 49 to 52 (December 2019), 8 district...","[12, 14, 0, 23, 7, 5]"
5059,2,2019-12-16,6,0.0,"From week 49 to 52 (December 2019), 8 district...","[16, 21]"
5060,4,2019-12-16,13,3.0,"From week 49 to 52 (December 2019), 8 district...","[3, 19, 18, 16, 22]"
5061,5,2019-12-16,9,0.0,"From week 49 to 52 (December 2019), 8 district...","[7, 12, 1]"
5062,7,2019-12-16,34,2.0,"From week 49 to 52 (December 2019), 8 district...","[8, 5, 1, 23]"
5063,8,2019-12-16,33,1.0,"From week 49 to 52 (December 2019), 8 district...","[5, 9, 12, 17, 20]"
5064,10,2019-12-16,0,0.0,"From week 49 to 52 (December 2019), 8 district...",[17]
5065,12,2019-12-16,44,0.0,"From week 49 to 52 (December 2019), 8 district...","[14, 1, 5, 8, 13, 17]"
5066,13,2019-12-16,0,0.0,"From week 49 to 52 (December 2019), 8 district...","[12, 17]"


In [8]:
data.loc[0:54]["Country"].unique()

array([ 0,  1,  5, 12, 14, 16, 22, 23,  7])

### Convert Weekly Data into Montly to Analyze Distributions

In [9]:
def update_batch(temp_batch, curr_batch):
    for index, row in curr_batch.iterrows():
            #check if the country is in the temp monthly batch, if not add it, otherwise update Cases, Deaths, and Summary
            if row["Country"] not in temp_batch["Country"].unique():
                if len(temp_batch) != 0:
                    row["Date"] = temp_batch["Date"].iloc[0]
                temp_batch = temp_batch.append(row)
            else:
                line_separator = "." if temp_batch.loc[temp_batch.Country == row["Country"], "Summary"].item()[-1] != "." else ""
                temp_batch.loc[temp_batch.Country == row["Country"], "Cases"] += row["Cases"]
                temp_batch.loc[temp_batch.Country == row["Country"], "Deaths"] += row["Deaths"]
                temp_batch.loc[temp_batch.Country == row["Country"], "Summary"] += line_separator + row["Summary"]
    return temp_batch
    
#Convert the weekly data into montly data (by summing over the Cases, Deaths, and concatenating the summaries)
count = 0
new_monthly_data = pd.DataFrame(columns=["Country", "Date", "Cases", "Deaths", "Summary", "Bordering Countries"])
#create a temporary dataframe for current month which will be concatenated to the final dataframe
temp_batch = new_monthly_data.copy()
curr_month = 1
#The upper-bound is determined by the number which when passed to the find_next_batch function will return the final batch
while count <= 5057:
    curr_batch = find_next_batch(count)
    count += len(curr_batch)
    #if the new batch is of a different month, add it to the new_monthly_data dataframe and empty the temp_batch for the new month
    if curr_batch["Date"].iloc[0].month != curr_month:
        curr_month = curr_batch["Date"].iloc[0].month
        new_monthly_data = pd.concat([new_monthly_data, temp_batch], ignore_index = True)
        temp_batch = pd.DataFrame(columns=["Country", "Date", "Cases", "Deaths", "Summary", "Bordering Countries"])
        #ensures last batch is added to the data
        if count >= 5057:
            new_monthly_data = pd.concat([new_monthly_data, find_next_batch(count)], ignore_index = True)
            break
    temp_batch = update_batch(temp_batch, curr_batch)

new_monthly_data.to_csv("../Current_Data/Month_data.csv", index=False)

    

1
2
3
4
5
6
7
9
10
11
12
13
14
16
17
18
19
20
22
23
24
26
27
28
29
30
31
33
34
35
36
37
38
40
41
42
43
44
45
47
48
49
50
52
53
54
55
56
58
59
60
61
63
64
65
66
67
68
69
71
72
73
74
75
76
78
79
80
81
82
84
85
86
87
88
90
91
92
93
94
95
96
98
99
100
101
102
104
105
106
108
109
110
112
113
114
116
117
118
120
121
122
123
125
126
127
128
129
131
132
133
134
135
137
138
139
141
142
143
144
145
147
148
149
150
151
152
154
155
156
158
159
160
161
162
163
165
166
167
168
170
171
172
173
175
176
177
178
179
180
181
182
184
185
186
187
189
190
191
192
194
195
196
198
199
200
201
202
203
204
206
207
208
209
210
211
212
213
215
216
217
218
219
220
221
222
223
225
226
227
228
229
230
231
232
234
235
236
237
238
239
240
242
243
244
245
246
247
248
250
251
252
253
254
255
256
257
259
260
261
262
263
264
265
266
268
269
270
271
272
273
274
275
277
278
279
280
281
282
283
284
286
287
288
289
290
291
292
293
294
296
297
298
299
300
301
302
304
305
306
307
308
309
311
312
313
314
316
317
318
319
320
322


2143
2144
2145
2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2157
2158
2160
2161
2162
2163
2164
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2175
2177
2178
2179
2180
2181
2182
2183
2184
2185
2186
2187
2188
2189
2190
2191
2192
2194
2195
2196
2197
2198
2199
2200
2201
2202
2203
2204
2205
2206
2207
2209
2210
2211
2212
2213
2214
2215
2216
2217
2218
2219
2220
2221
2222
2223
2225
2226
2227
2228
2229
2230
2231
2232
2233
2234
2235
2236
2238
2239
2240
2241
2242
2243
2244
2245
2246
2247
2248
2249
2250
2251
2253
2254
2255
2256
2257
2258
2259
2260
2261
2263
2264
2265
2266
2267
2268
2269
2270
2271
2272
2273
2275
2276
2277
2278
2279
2280
2281
2282
2283
2285
2286
2287
2288
2289
2290
2291
2292
2293
2294
2296
2297
2298
2299
2300
2301
2302
2303
2304
2305
2306
2308
2309
2310
2311
2312
2313
2314
2315
2316
2317
2318
2319
2321
2322
2323
2324
2325
2326
2327
2328
2329
2330
2331
2332
2333
2335
2336
2337
2338
2339
2340
2341
2342
2343
2344
2345
2346
2348
2349
2350
2351
2352
2353
2354
2355
2356
2357


3963
3964
3965
3966
3967
3968
3969
3970
3971
3972
3973
3974
3975
3976
3977
3978
3979
3981
3982
3983
3984
3985
3986
3987
3988
3989
3990
3991
3992
3993
3994
3995
3997
3998
3999
4000
4001
4002
4003
4004
4005
4006
4007
4008
4009
4010
4012
4013
4014
4015
4016
4017
4018
4019
4020
4021
4022
4023
4024
4025
4026
4027
4028
4029
4030
4031
4033
4034
4035
4036
4037
4038
4039
4040
4041
4042
4044
4045
4046
4047
4048
4049
4050
4051
4052
4053
4054
4055
4056
4058
4059
4060
4061
4062
4063
4064
4065
4066
4067
4068
4069
4070
4071
4073
4074
4075
4076
4077
4078
4079
4080
4081
4082
4083
4084
4085
4086
4088
4089
4090
4091
4092
4093
4094
4095
4096
4097
4098
4099
4100
4101
4102
4104
4105
4106
4107
4108
4109
4110
4111
4112
4113
4114
4115
4116
4117
4118
4119
4121
4122
4123
4124
4125
4126
4127
4128
4129
4130
4131
4132
4133
4134
4135
4137
4138
4139
4140
4141
4142
4143
4144
4145
4146
4147
4148
4149
4150
4151
4152
4153
4155
4156
4157
4158
4159
4160
4161
4162
4163
4164
4165
4166
4167
4168
4169
4170
4171
4173
4174
4175


### Plotting Histograms With No. Of Cases for Each Country

In [10]:
"""This function will go through any time-series data in a pandas dataframe e.g. weekly, monthly cases etc. and 
return two dic with 
data_dict: countries as keys and a list of datapoints (weekly, monthly, yearly etc.) as the values
threshold_dict: countries as keys and list representing the 30th and 70th percentiles as values

"""

#This is an update get_thresholds method after realizing that the function below computed independent thresholds
#for each country for each year => we weren't getting an even distribution across all states
#commented out the code which computes the thresholds in case we need to go back to it
def get_thresholds(df):
    threshold_dict = {}
    for country in list(df["Country"].unique()):
        country_df = df.loc[df["Country"] == country]
        first_percentile = country_df["Cases"].quantile(0.3)
        second_percentile = country_df["Cases"].quantile(0.7)
        threshold_dict[country] = [first_percentile, second_percentile]
    return threshold_dict


def time_series_to_dict(df):
    data_dict = {}
    threshold_dict = {}
    for i in range(2005, 2020):
        curr_yr = df.loc[df.Date.dt.year == i]
        countries_list=list(curr_yr["Country"].unique())
        for country in countries_list:
            country_curr_yr = curr_yr[curr_yr.Country == country]
#             first_percentile = country_curr_yr["Cases"].quantile(0.3)
#             second_percentile =  country_curr_yr["Cases"].quantile(0.7)
#             threshold_dict[country] = [first_percentile, second_percentile]
            if country not in data_dict:
                data_dict[country] = country_curr_yr["Cases"].tolist()
            else:
                data_dict[country]+= country_curr_yr["Cases"].tolist()
#     if thresholds_needed:
#         return data_dict, threshold_dict
    return data_dict

monthly_data_dict = time_series_to_dict(new_monthly_data)


#Apply a normal test to see if the data for each country reflects a Gaussian distribution.
# A 5% significiance level is used for each country (and the null hypothesis is rejected if p-value <0.05)

# for key, values in monthly_data_dict.items():
#     if len(values) > 20:

#         plt.hist(values)
#         plt.show()
#         stat, p = normaltest(values)
#         print(country_list[key])
#         print('Statistics=%.3f, p=%.3f' % (stat, p))
#         # interpret
#         alpha = 0.05
#         if p > alpha:
#             print('Sample looks Gaussian (fail to reject H0)')
#         else:
#             print('Sample does not look Gaussian (reject H0)')
        
        

### Determining State Thresholds for Each Country 
To determine the threshold for each country (i.e. no. of cases for each country that will determine whether the severity is low, medium, or high), we will start by computing independent thresholds for each country. Since each country has different populations, factors affecting the spread of the pandemic (e.g. no. of people vaccinated etc.), the same number of cases in different locations has different implications (e.g. 10 cases for a population of 100 is much more dangerous than 10 for 1,000,000. The thresholds for low, medium, and high will be the values that account fo 30%, 40%, and 30% respectively. In other words, each country will have two thresholds at the 30th and 70th and percentiles (calculated from the time_series_to_dict function above).

0 = low  

1 = medium 

2 = high

In [11]:
#what to do data with thresholds of 0,0? For now drop
thresholds_weekly_data = get_thresholds(data)
for country, thresholds in thresholds_weekly_data.items():
    #*NOTE updated thresholds which are now ignored to include 0 and 1 or 0 and 2
    if (thresholds[0] == 0 and thresholds[1] == 0) or (thresholds[0] == 0 and thresholds[1] <= 2):
        indexNames = data[data['Country'] == country ].index
        data = data.drop(indexNames, axis=0)
        print(country)

weekly_data_dict = time_series_to_dict(data)
thresholds_weekly_data = get_thresholds(data)

data["State"] = 0
def find_state(country_threshold_list, no_cases):
    if no_cases >= float(country_threshold_list[1]):
        return 2
    elif no_cases >= float(country_threshold_list[0]):
        return 1
    else:
        return 0


#find the state for each country by comparing the number of cases to the corresponding thresholds 
data["State"] = data.apply(lambda x: find_state(thresholds_weekly_data[x["Country"]], x["Cases"]) ,axis=1)
data.head()
data = data.reset_index()
# scaler = MinMaxScaler()
# scaled_data = data.copy()
# scaled_data[["Cases", "Deaths"]] = scaler.fit_transform(scaled_data[["Cases", "Deaths"]])
# scaled_data.head()
# weekly_data_dict_scaled = time_series_to_dict(scaled_data)
#thresholds_weekly_data_scaled = get_thresholds(scaled_data)
print(weekly_data_dict)

10
13
20
21
9
{0: [13, 7, 13, 14, 13, 10, 5, 4, 3, 2, 2, 0, 4, 12, 13, 14, 5, 3, 10, 9, 14, 10, 1, 2, 4, 7, 26, 4, 12, 12, 23, 22, 27, 10, 12, 17, 44, 3, 4, 19, 7, 16, 9, 42, 57, 14, 14, 18, 17, 16, 18, 23, 4, 19, 5, 2, 18, 8, 6, 6, 3, 5, 4, 4, 4, 4, 9, 21, 19, 5, 19, 16, 33, 11, 11, 11, 66, 11, 15, 5, 13, 14, 4, 4, 3, 1, 3, 1, 3, 2, 4, 1, 0, 21, 20, 20, 20, 39, 12, 11, 5, 4, 22, 13, 13, 13, 15, 8, 11, 8, 12, 16, 16, 7, 4, 5, 4, 5, 5, 6, 2, 3, 7, 14, 16, 8, 8, 14, 8, 10, 11, 9, 150, 9, 7, 10, 14, 12, 166, 6, 5, 11, 1, 5, 6, 7, 4, 1, 2, 2, 1, 6, 0, 16, 14, 9, 19, 11, 8, 9, 21, 33, 31, 30, 65, 73, 45, 64, 61, 90, 66, 51, 26, 15, 22, 20, 12, 7, 7, 10, 8, 8, 99, 59, 55, 43, 21, 14, 21, 26, 25, 26, 26, 38, 38, 35, 24, 15, 14, 18, 11, 12, 11, 8, 17, 16, 21, 12, 8, 20, 47, 66, 51, 67, 46, 48, 11, 14, 12, 18, 15, 25, 20, 17, 24, 15, 20, 16, 17, 11, 9, 16, 15, 12, 14, 17, 20, 18, 16, 14, 17, 14, 65, 63, 37, 57, 44, 28, 9, 5, 7, 5, 8, 5, 6, 5, 11, 10, 9, 6, 7, 10, 11, 10, 8, 0, 5, 6, 9, 7, 12, 1

In [13]:
#As a quick note - all of the LSTM tests were done on the old threshold data - have not corrected to see if the 
#results changed but I suspect there would be no difference
test = []
burkina_faso_data = []
def return_features_labels(window_size, weekly_dict, thresholds, state=True):
    #input data is an array of tuples (features, label) where features is the sequence data of no. of cases and label 
#is the state
    input_data = []
    #array of tuples of (features, label) where label is also a tuple of (country, no. of case at t+1, state at t+1)
    continuous_data = []
    for country, cases in weekly_dict.items():
        for i in range(len(cases)-window_size-1):
            features = cases[i:i+window_size]
            label = find_state(thresholds[country], cases[i+1+window_size])
            input_data.append((features, label))
            continuous_data.append((features, (country, cases[i+1+window_size], label)))
            if country == 1 and window_size==4:
                burkina_faso_data.append((features, (country, cases[i+1+window_size], label)))
    if state == False:
        return continuous_data
    return input_data
#uncommented to so burkina_faso data won't work
# a = return_features_labels(4, weekly_data_dict, thresholds_weekly_data)
input_data_4 = return_features_labels(4, weekly_data_dict, thresholds_weekly_data)
input_data_5 = return_features_labels(5, weekly_data_dict, thresholds_weekly_data, state=False)
input_data_6 = return_features_labels(6, weekly_data_dict, thresholds_weekly_data, state=False)
input_data_7 = return_features_labels(7, weekly_data_dict, thresholds_weekly_data, state=False)
continuous_data = return_features_labels(4, weekly_data_dict, thresholds_weekly_data, state = False)
continuous_data_10 = return_features_labels(10, weekly_data_dict, thresholds_weekly_data, state=False)
print(continuous_data_10)
# input_data_4_scaled = return_features_labels(4, weekly_data_dict_scaled, thresholds_weekly_data_scaled)

[([13, 7, 13, 14, 13, 10, 5, 4, 3, 2], (0, 0, 0)), ([7, 13, 14, 13, 10, 5, 4, 3, 2, 2], (0, 4, 0)), ([13, 14, 13, 10, 5, 4, 3, 2, 2, 0], (0, 12, 1)), ([14, 13, 10, 5, 4, 3, 2, 2, 0, 4], (0, 13, 1)), ([13, 10, 5, 4, 3, 2, 2, 0, 4, 12], (0, 14, 1)), ([10, 5, 4, 3, 2, 2, 0, 4, 12, 13], (0, 5, 0)), ([5, 4, 3, 2, 2, 0, 4, 12, 13, 14], (0, 3, 0)), ([4, 3, 2, 2, 0, 4, 12, 13, 14, 5], (0, 10, 1)), ([3, 2, 2, 0, 4, 12, 13, 14, 5, 3], (0, 9, 1)), ([2, 2, 0, 4, 12, 13, 14, 5, 3, 10], (0, 14, 1)), ([2, 0, 4, 12, 13, 14, 5, 3, 10, 9], (0, 10, 1)), ([0, 4, 12, 13, 14, 5, 3, 10, 9, 14], (0, 1, 0)), ([4, 12, 13, 14, 5, 3, 10, 9, 14, 10], (0, 2, 0)), ([12, 13, 14, 5, 3, 10, 9, 14, 10, 1], (0, 4, 0)), ([13, 14, 5, 3, 10, 9, 14, 10, 1, 2], (0, 7, 0)), ([14, 5, 3, 10, 9, 14, 10, 1, 2, 4], (0, 26, 2)), ([5, 3, 10, 9, 14, 10, 1, 2, 4, 7], (0, 4, 0)), ([3, 10, 9, 14, 10, 1, 2, 4, 7, 26], (0, 12, 1)), ([10, 9, 14, 10, 1, 2, 4, 7, 26, 4], (0, 12, 1)), ([9, 14, 10, 1, 2, 4, 7, 26, 4, 12], (0, 23, 2)), ([14, 10,

In [14]:
for country, values in weekly_data_dict.items():
    print(country_list[country], " : ", len(values))

Benin  :  399
Burkina Faso  :  425
Ivory Coast  :  335
Mali  :  403
Niger  :  380
Dem. Rep. Congo  :  195
Chad  :  303
Togo  :  366
Ghana  :  264
Nigeria  :  345
Ethiopia  :  102
Centrafrique  :  212
Guinea  :  190
Cameroun  :  168
Sudan  :  175
Senegal  :  221
South Sudan  :  100
Kenya  :  34
Uganda  :  19
Burundi  :  22


Above is a list of all of the countries being included in the final input_data that is passed into the LSTM. The number of recorded datapoints for different countries seems to be extremely disproportionate. This means that once source of error when testing is that the data comes from a different distribution to the training data (i.e. we train on data for example only on Burkina Faso and the test data contains data for Uganda).

One potential problem:

When I collect data, and put into a large array [1, 0, 2] I assume that each datapoint is the recorded number of cases for the week after the first one. In practice, this may not necessarily be the case for all the datapoints (could be months or even years apart) - how do I handle this?

Also difference in temporal time frame between inital weekly and later monthly data (sequence are representing different temporal timeframes)

Strategies for handling missing data
1. Only use data without gaps (as I have done)
2. Fill in the gaps (e.g. with the last observed value or the median)


### Monthly Data
We are gonna test the LSTM on monthly data but we are going to do a couple of things differently this time
1. We are gonna impute missing values by plotting a regression curve with the points we have. This will ensure the temporal difference between points is constant (which would otherwise increase the amount of noise in an already noisy dataset)

In [14]:
new_monthly_data = pd.read_csv("../Current_Data/Month_data.csv", usecols=['Country','Date','Cases','Deaths','Summary'],  
parse_dates=["Date"])
monthly_thresholds_all = get_thresholds(new_monthly_data)
for country, thresholds in monthly_thresholds_all.items():
    #*NOTE updated thresholds which are now ignored to include 0 and 1 or 0 and 2
    if (thresholds[0] == 0 and thresholds[1] == 0) or (thresholds[0] == 0 and thresholds[1] <= 2):
        indexNames = new_monthly_data[new_monthly_data['Country'] == country ].index
        new_monthly_data = new_monthly_data.drop(indexNames, axis=0)
#remove any data from 2005 since there are only 4 months
new_monthly_data = new_monthly_data.loc[new_monthly_data["Date"].dt.year != 2005]
new_monthly_data = new_monthly_data.set_index("Date", drop=True)
all_dates = list(pd.date_range("2006-1-31", periods=12*14, freq="M").values)
print(monthly_thresholds_all.keys())

dict_keys([0, 1, 5, 12, 14, 16, 22, 23, 7, 15, 6, 4, 8, 3, 19, 10, 17, 13, 18, 11, 20, 21, 9, 24, 2])


In [59]:
month_data = pd.DataFrame(columns=["Date", "Country", "Cases"])
#impute missing values
#start with these countries to see if it will work 
for country in [1,12,14,22,5,16,23,0]:
    country_df = new_monthly_data.loc[new_monthly_data["Country"] == country]
    missing_dates = []
    #resample so every date becomes last possible day of its month
    country_df = country_df.resample("M").sum()
    country_df = country_df.reset_index()
    #The code below creates a dataframe for each country that contains its initial datapoints and adds the dates
    #with missing datapoints as nan. This is to ensure all dataframes have the same size since we will be imputing
    #missing values
    filled_in_df = pd.DataFrame(columns=['Date', 'Cases'])
    for j in range(len(all_dates)):
        date = all_dates[j]
        if date not in country_df["Date"].values:
            filled_in_df.loc[j] = [date] + [np.NaN]
            missing_dates.append(date)
        else:
            filled_in_df.loc[j] = [date] + [float(country_df.loc[country_df.Date == date]["Cases"].values[0])]
    
     #convert dates to ordinal so regression can work more effectively
    country_df["Date"].map(datetime.datetime.toordinal)
    #impute missing values by plotting regression curve and finding missing values from curve
    if len(country_df) != len(all_dates):
        model = KernelRidge()
        model.fit(country_df["Date"].values.reshape(-1, 1), country_df["Cases"].values.reshape(-1, 1))
        for date in missing_dates:
            ordinal_date = datetime.datetime.toordinal(date)
            input_prediction = np.array(ordinal_date).reshape(1, -1)
            imputed_calculation = int(model.predict(input_prediction)[0][0])
            filled_in_df.loc[filled_in_df["Date"] == date, "Cases"] = imputed_calculation
    filled_in_df["Country"] = country
#     sum_cases = filled_in_df["Cases"].sum()
#     mean = filled_in_df["Cases"].mean()
#     std = filled_in_df["Cases"].std()
#     filled_in_df["Cases"] = filled_in_df["Cases"].apply(lambda x: (x-mean)/std)
    month_data = pd.concat((month_data, filled_in_df))

monthly_thresholds = get_thresholds(month_data)
month_data["State"] = month_data.apply(lambda x: find_state(monthly_thresholds[x["Country"]], x["Cases"]) ,axis=1)
new_monthly_data_dict = time_series_to_dict(month_data)
month_data.to_csv("updated_monthly_data.csv", index=False, header=False)


### Prepare Monthly Data for Natural Language Processing Task

In [15]:

new_monthly_data["State"] = new_monthly_data.apply(lambda x: find_state(monthly_thresholds_all[x["Country"]], 
                                                   x["Cases"]), axis=1)
new_monthly_data = new_monthly_data.reset_index()
#Countries for which there is available monthly data => label will be array of length 20
countries_monthly = np.array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 14, 15, 16, 17, 18,
       22, 23, 24])

#this function will take a dataframe that contains either weekly or monthly datapoints and convert into a 
#dataframe that can be used by a language model like BERT/Longformer for multilabel classification
def prepare_df_to_lang(df):
    #This dataframe will contains individual samples of the summary with just a summary and the combined 
    #representative state - state represents the sum of the states of the countries in each report while count 
    #represents the number of countries
    summary_state = pd.DataFrame(columns=["Summary", "State", "Count", "Label"])
    #used for classification of state from text
    multi_label_classification = pd.DataFrame(columns=["text", "labels"])
    #prepare multilabel classification the t_labels represent the labels at the current timestep
    #while t+1_labels are the labels at the next timestep
    bert_threshold_classification = pd.DataFrame(columns=["text", "t_labels", "t+1_labels"])
    
    state_total = 0
    state_count = 0
    summary = df.iloc[0]["Summary"]
    total = len(summary.split())
    curr_date = df.iloc[0]["Date"]
    max_length = 0
    #array of length 20 that will contain the state for each country at each specific index e.g. 0 benin 1 bukinra
    #would be [0, 1 etc..]
    curr_label = np.zeros(20)
    
    #array of length 20 that will contain the no. of cases for each country at the current timestep (if the value
    #at a specific index is -1, then there is not data for that country available)
    curr_cases = np.full((20), -1)
    future_cases = np.full((20), -1)
    
    #index to keep track of next batch where batch = all of the datapoints with the same date at the next timestep
    #will get the first index of the next batch
    start_index = find_next_batch(0, df).index[-1]+1
    for index, row in df.iterrows():
        if row["Date"] != curr_date:
            #saved current data before resetting it
            summary_state.loc[index] = [summary] + [state_total] + [state_count] + [round(state_total/state_count)]
            multi_label_classification.loc[index] = [summary] + [curr_label] 
            bert_threshold_classification.loc[index] = [summary] + [curr_cases] + [future_cases]
            #NOTE: would be faster to set the curr_cases to future_cases but in case there are additional datapoints
            #in the next batch that weren't captured in future_cases (since future_cases will only record future values
            #of datapoints at timestep t), we will repeatedly populate curr_cases
            
            #reset the data
            curr_label = np.zeros(20)
            state_count = 1
            state_total = row["State"]
            summary = row["Summary"]
            curr_date = row["Date"]
            
            #reset BERT threshold data
            curr_cases = np.full((20), -1)
            future_cases = np.full((20), -1)
            #need to get the first index of the new batch to populate the future_cases
            start_index = find_next_batch(index, df).index[-1]+1
            
            total += len(summary.split())
        else:
            state_count += 1
            state_total += row["State"]
            max_length = len(summary.split()) if len(summary.split()) > max_length else max_length
            #access correct index of curr_label by checking corresponding index from countries_monthly
            curr_index = np.where(countries_monthly==row["Country"])
            
            curr_cases[curr_index] = row["Cases"]
            #needs to be in a try block since there will be no future cases to retreive for the last batch
            try:
                print(index)
                next_batch = find_next_batch(start_index, df)
                next_cases = next_batch.loc[next_batch["Country"] == row["Country"], "Cases"].item()
                future_cases[curr_index] = next_cases
            except Exception as e:
                print(e)
            
            curr_label[curr_index] = row["State"] #if row["State"] == 1 or row["State"] == 2 else 0
    return (summary_state, multi_label_classification, bert_threshold_classification)

#TODO: check summary being taken is the correct one since rn we are using the summary of the first item in the batch
#however these are not always the same with the rest of the items in the batch
_, multi_label_classification, bert_threshold_classification = prepare_df_to_lang(new_monthly_data)
multi_label_classification.to_csv("Text Information Extraction/multilabel_classification.csv", index=False)
multi_label_classification

1
2
3
4
5
6
7
0
9
10
11
12
13
14
15
16
1
9
10
11
12
13
14
15
16
2
9
10
11
12
13
14
15
16
3
9
10
11
12
13
14
15
16
4
9
10
11
12
13
14
15
16
can only convert an array of size 1 to a Python scalar
5
9
10
11
12
13
14
15
16
6
9
10
11
12
13
14
15
16
7
9
10
11
12
13
14
15
16
can only convert an array of size 1 to a Python scalar
9
10
11
12
13
14
15
16
9
18
19
20
21
22
23
24
25
10
18
19
20
21
22
23
24
25
11
18
19
20
21
22
23
24
25
12
18
19
20
21
22
23
24
25
can only convert an array of size 1 to a Python scalar
13
18
19
20
21
22
23
24
25
14
18
19
20
21
22
23
24
25
can only convert an array of size 1 to a Python scalar
15
18
19
20
21
22
23
24
25
16
18
19
20
21
22
23
24
25
can only convert an array of size 1 to a Python scalar
18
19
20
21
22
23
24
25
18
27
28
29
30
31
32
33
34
19
27
28
29
30
31
32
33
34
can only convert an array of size 1 to a Python scalar
20
27
28
29
30
31
32
33
34
21
27
28
29
30
31
32
33
34
22
27
28
29
30
31
32
33
34
23
27
28
29
30
31
32
33
34
24
27
28
29
30
31
32
33
34
can o

235
243
244
245
246
247
248
249
250
251
252
253
236
243
244
245
246
247
248
249
250
251
252
253
237
243
244
245
246
247
248
249
250
251
252
253
238
243
244
245
246
247
248
249
250
251
252
253
239
243
244
245
246
247
248
249
250
251
252
253
240
243
244
245
246
247
248
249
250
251
252
253
241
243
244
245
246
247
248
249
250
251
252
253
243
244
245
246
247
248
249
250
251
252
253
243
255
256
257
258
259
260
261
262
263
264
244
255
256
257
258
259
260
261
262
263
264
245
255
256
257
258
259
260
261
262
263
264
can only convert an array of size 1 to a Python scalar
246
255
256
257
258
259
260
261
262
263
264
247
255
256
257
258
259
260
261
262
263
264
248
255
256
257
258
259
260
261
262
263
264
249
255
256
257
258
259
260
261
262
263
264
250
255
256
257
258
259
260
261
262
263
264
251
255
256
257
258
259
260
261
262
263
264
252
255
256
257
258
259
260
261
262
263
264
253
255
256
257
258
259
260
261
262
263
264
255
256
257
258
259
260
261
262
263
264
255
266
267
268
269
270
271
272
273
256
2

431
443
444
445
446
447
448
449
450
451
452
453
454
432
443
444
445
446
447
448
449
450
451
452
453
454
433
443
444
445
446
447
448
449
450
451
452
453
454
434
443
444
445
446
447
448
449
450
451
452
453
454
435
443
444
445
446
447
448
449
450
451
452
453
454
436
443
444
445
446
447
448
449
450
451
452
453
454
437
443
444
445
446
447
448
449
450
451
452
453
454
438
443
444
445
446
447
448
449
450
451
452
453
454
439
443
444
445
446
447
448
449
450
451
452
453
454
440
443
444
445
446
447
448
449
450
451
452
453
454
441
443
444
445
446
447
448
449
450
451
452
453
454
443
444
445
446
447
448
449
450
451
452
453
454
443
456
457
458
459
460
461
462
463
464
465
444
456
457
458
459
460
461
462
463
464
465
445
456
457
458
459
460
461
462
463
464
465
446
456
457
458
459
460
461
462
463
464
465
447
456
457
458
459
460
461
462
463
464
465
can only convert an array of size 1 to a Python scalar
448
456
457
458
459
460
461
462
463
464
465
449
456
457
458
459
460
461
462
463
464
465
can only convert 

597
598
599
600
601
602
603
604
605
606
597
608
609
610
611
612
613
614
615
616
617
618
598
608
609
610
611
612
613
614
615
616
617
618
599
608
609
610
611
612
613
614
615
616
617
618
600
608
609
610
611
612
613
614
615
616
617
618
601
608
609
610
611
612
613
614
615
616
617
618
602
608
609
610
611
612
613
614
615
616
617
618
603
608
609
610
611
612
613
614
615
616
617
618
604
608
609
610
611
612
613
614
615
616
617
618
605
608
609
610
611
612
613
614
615
616
617
618
606
608
609
610
611
612
613
614
615
616
617
618
can only convert an array of size 1 to a Python scalar
608
609
610
611
612
613
614
615
616
617
618
608
620
621
622
623
624
625
626
627
628
629
609
620
621
622
623
624
625
626
627
628
629
can only convert an array of size 1 to a Python scalar
610
620
621
622
623
624
625
626
627
628
629
611
620
621
622
623
624
625
626
627
628
629
612
620
621
622
623
624
625
626
627
628
629
613
620
621
622
623
624
625
626
627
628
629
614
620
621
622
623
624
625
626
627
628
629
615
620
621
622
62

757
762
763
764
765
766
767
768
769
770
771
772
773
758
762
763
764
765
766
767
768
769
770
771
772
773
can only convert an array of size 1 to a Python scalar
759
762
763
764
765
766
767
768
769
770
771
772
773
760
762
763
764
765
766
767
768
769
770
771
772
773
762
763
764
765
766
767
768
769
770
771
772
773
762
775
776
777
778
779
780
781
782
783
784
785
786
763
775
776
777
778
779
780
781
782
783
784
785
786
764
775
776
777
778
779
780
781
782
783
784
785
786
765
775
776
777
778
779
780
781
782
783
784
785
786
766
775
776
777
778
779
780
781
782
783
784
785
786
can only convert an array of size 1 to a Python scalar
767
775
776
777
778
779
780
781
782
783
784
785
786
768
775
776
777
778
779
780
781
782
783
784
785
786
769
775
776
777
778
779
780
781
782
783
784
785
786
770
775
776
777
778
779
780
781
782
783
784
785
786
771
775
776
777
778
779
780
781
782
783
784
785
786
772
775
776
777
778
779
780
781
782
783
784
785
786
773
775
776
777
778
779
780
781
782
783
784
785
786
775
776
77

...

1724
1735
1736
1737
1738
1739
1740
1741
1742
1743
1744
1745
1746
1747
1748
1749
1750
1751
1725
1735
1736
1737
1738
1739
1740
1741
1742
1743
1744
1745
1746
1747
1748
1749
1750
1751
1726
1735
1736
1737
1738
1739
1740
1741
1742
1743
1744
1745
1746
1747
1748
1749
1750
1751
1727
1735
1736
1737
1738
1739
1740
1741
1742
1743
1744
1745
1746
1747
1748
1749
1750
1751
1728
1735
1736
1737
1738
1739
1740
1741
1742
1743
1744
1745
1746
1747
1748
1749
1750
1751
1729
1735
1736
1737
1738
1739
1740
1741
1742
1743
1744
1745
1746
1747
1748
1749
1750
1751
1730
1735
1736
1737
1738
1739
1740
1741
1742
1743
1744
1745
1746
1747
1748
1749
1750
1751
1731
1735
1736
1737
1738
1739
1740
1741
1742
1743
1744
1745
1746
1747
1748
1749
1750
1751
1732
1735
1736
1737
1738
1739
1740
1741
1742
1743
1744
1745
1746
1747
1748
1749
1750
1751
1733
1735
1736
1737
1738
1739
1740
1741
1742
1743
1744
1745
1746
1747
1748
1749
1750
1751
1735
1736
1737
1738
1739
1740
1741
1742
1743
1744
1745
1746
1747
1748
1749
1750
1751
1735
1753
1754


can only convert an array of size 1 to a Python scalar
1852
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1853
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1854
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1855
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1856
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1858
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1859
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1860
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1861
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1862
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885


1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
1992
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
1993
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
1994
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
can only convert an array of size 1 to a Python scalar
1995
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
1996
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
1997
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
1998
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
1999
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2000
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2001
2011
2012
2013
2014
2015
2016
2017
2018


2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2115
2130
2131
2116
2130
2131
2117
2130
2131
2118
2130
2131
2119
2130
2131
2120
2130
2131
2121
2130
2131
2122
2130
2131
2123
2130
2131
2124
2130
2131
2125
2130
2131
2126
2130
2131
2127
2130
2131
2128
2130
2131


,text,labels
8,The districts of Guidan-Roumji in Niger and Do...,"[0.0, 2.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
17,The week 6 was marked by a sudden apparition o...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
26,Spatial analysis shows an epidemic area common...,"[0.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 2.0, ..."
35,During week 14 it is noted that: Burkina Faso:...,"[0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 1.0, 0.0, ..."
44,Burkina Faso: Titao was the last district in a...,"[0.0, 1.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, ..."
...,...,...
2046,"At week 23 of year 2019, no district reached t...","[0.0, 0.0, 1.0, 0.0, 2.0, 0.0, 0.0, 1.0, 1.0, ..."
2064,"From week 27 to 30 (July 2019), no district re...","[0.0, 0.0, 1.0, 0.0, 2.0, 0.0, 0.0, 1.0, 2.0, ..."
2082,"From week 31 to 35 (August 2019), no district ...","[0.0, 0.0, 2.0, 0.0, 2.0, 0.0, 0.0, 1.0, 1.0, ..."
2097,"From week 36 to 39 (September 2019), of the co...","[0.0, 0.0, 2.0, 0.0, 2.0, 0.0, 0.0, 0.0, 1.0, ..."


In [94]:
#Preparing weekly data to see if there will be a difference in the multilabel classification accuracy
weekly_data_lang = data.loc[data["Country"] == 1]
weekly_data_lang = weekly_data_lang.drop(columns=["Bordering Countries", "Cases", "Deaths", "Country"])
_, multi_label_classification_weekly, bert_threshold_weekly = prepare_df_to_lang(data)
# multi_label_classification_weekly.to_csv("Text Information Extraction/multilabel_classification_weekly.csv", index=False)
bert_threshold_weekly.to_csv("Text Information Extraction/threshold_weekly_data.csv", index=False)



1
2
3
4
5
6
7
0
9
10
11
12
13
14
1
9
10
11
12
13
14
2
9
10
11
12
13
14
3
9
10
11
12
13
14
4
9
10
11
12
13
14
5
9
10
11
12
13
14
6
9
10
11
12
13
14
can only convert an array of size 1 to a Python scalar
7
9
10
11
12
13
14
9
10
11
12
13
14
9
16
17
18
19
20
10
16
17
18
19
20
11
16
17
18
19
20
12
16
17
18
19
20
13
16
17
18
19
20
14
16
17
18
19
20
can only convert an array of size 1 to a Python scalar
16
17
18
19
20
16
22
23
24
can only convert an array of size 1 to a Python scalar
17
22
23
24
18
22
23
24
can only convert an array of size 1 to a Python scalar
19
22
23
24
20
22
23
24
22
23
24
22
26
27
28
29
30
31
23
26
27
28
29
30
31
can only convert an array of size 1 to a Python scalar
24
26
27
28
29
30
31
26
27
28
29
30
31
26
33
34
35
36
37
38
27
33
34
35
36
37
38
can only convert an array of size 1 to a Python scalar
28
33
34
35
36
37
38
29
33
34
35
36
37
38
30
33
34
35
36
37
38
31
33
34
35
36
37
38
33
34
35
36
37
38
33
40
41
42
43
44
45
34
40
41
42
43
44
45
35
40
41
42
43
44
45
can only

298
299
300
301
302
292
296
297
298
299
300
301
302
can only convert an array of size 1 to a Python scalar
293
296
297
298
299
300
301
302
294
296
297
298
299
300
301
302
can only convert an array of size 1 to a Python scalar
296
297
298
299
300
301
302
296
304
305
306
307
308
309
297
304
305
306
307
308
309
298
304
305
306
307
308
309
can only convert an array of size 1 to a Python scalar
299
304
305
306
307
308
309
300
304
305
306
307
308
309
301
304
305
306
307
308
309
302
304
305
306
307
308
309
304
305
306
307
308
309
304
311
312
313
314
305
311
312
313
314
can only convert an array of size 1 to a Python scalar
306
311
312
313
314
307
311
312
313
314
308
311
312
313
314
309
311
312
313
314
311
312
313
314
311
316
317
318
319
320
312
316
317
318
319
320
can only convert an array of size 1 to a Python scalar
313
316
317
318
319
320
can only convert an array of size 1 to a Python scalar
314
316
317
318
319
320
316
317
318
319
320
316
322
323
324
325
326
327
328
317
322
323
324
325
32

536
538
539
540
541
542
543
544
545
538
539
540
541
542
543
544
545
538
547
548
549
550
551
552
539
547
548
549
550
551
552
540
547
548
549
550
551
552
can only convert an array of size 1 to a Python scalar
541
547
548
549
550
551
552
can only convert an array of size 1 to a Python scalar
542
547
548
549
550
551
552
543
547
548
549
550
551
552
544
547
548
549
550
551
552
545
547
548
549
550
551
552
547
548
549
550
551
552
547
554
555
556
557
558
559
560
548
554
555
556
557
558
559
560
549
554
555
556
557
558
559
560
550
554
555
556
557
558
559
560
551
554
555
556
557
558
559
560
552
554
555
556
557
558
559
560
554
555
556
557
558
559
560
554
562
563
564
565
566
567
568
555
562
563
564
565
566
567
568
556
562
563
564
565
566
567
568
557
562
563
564
565
566
567
568
can only convert an array of size 1 to a Python scalar
558
562
563
564
565
566
567
568
559
562
563
564
565
566
567
568
560
562
563
564
565
566
567
568
562
563
564
565
566
567
568
562
570
571
572
573
574
575
576
563
570
571
572

793
800
801
802
803
804
805
806
can only convert an array of size 1 to a Python scalar
794
800
801
802
803
804
805
806
795
800
801
802
803
804
805
806
796
800
801
802
803
804
805
806
797
800
801
802
803
804
805
806
798
800
801
802
803
804
805
806
800
801
802
803
804
805
806
800
808
809
810
811
812
813
814
815
816
801
808
809
810
811
812
813
814
815
816
802
808
809
810
811
812
813
814
815
816
803
808
809
810
811
812
813
814
815
816
804
808
809
810
811
812
813
814
815
816
805
808
809
810
811
812
813
814
815
816
806
808
809
810
811
812
813
814
815
816
808
809
810
811
812
813
814
815
816
808
818
819
820
821
822
823
824
825
809
818
819
820
821
822
823
824
825
can only convert an array of size 1 to a Python scalar
810
818
819
820
821
822
823
824
825
811
818
819
820
821
822
823
824
825
812
818
819
820
821
822
823
824
825
813
818
819
820
821
822
823
824
825
814
818
819
820
821
822
823
824
825
815
818
819
820
821
822
823
824
825
816
818
819
820
821
822
823
824
825
818
819
820
821
822
823
824
82

1022
1023
1024
1025
1026
1027
1022
1029
1030
1031
1032
1033
1034
1035
1036
1037
1023
1029
1030
1031
1032
1033
1034
1035
1036
1037
1024
1029
1030
1031
1032
1033
1034
1035
1036
1037
1025
1029
1030
1031
1032
1033
1034
1035
1036
1037
1026
1029
1030
1031
1032
1033
1034
1035
1036
1037
1027
1029
1030
1031
1032
1033
1034
1035
1036
1037
1029
1030
1031
1032
1033
1034
1035
1036
1037
1029
1039
1040
1041
1042
1043
1044
1045
1046
1047
1030
1039
1040
1041
1042
1043
1044
1045
1046
1047
1031
1039
1040
1041
1042
1043
1044
1045
1046
1047
can only convert an array of size 1 to a Python scalar
1032
1039
1040
1041
1042
1043
1044
1045
1046
1047
1033
1039
1040
1041
1042
1043
1044
1045
1046
1047
1034
1039
1040
1041
1042
1043
1044
1045
1046
1047
1035
1039
1040
1041
1042
1043
1044
1045
1046
1047
1036
1039
1040
1041
1042
1043
1044
1045
1046
1047
1037
1039
1040
1041
1042
1043
1044
1045
1046
1047
1039
1040
1041
1042
1043
1044
1045
1046
1047
1039
1049
1050
1051
1052
1053
1054
1055
1056
1057
1040
1049
1050
1051
1052


...

4074
4076
4077
4078
4079
4080
4081
4082
4083
4084
4085
4086
4087
4088
4076
4077
4078
4079
4080
4081
4082
4083
4084
4085
4086
4087
4088
4076
4090
4091
4092
4093
4094
4095
4096
4097
4098
4099
4100
4101
4102
4103
4104
4105
4077
4090
4091
4092
4093
4094
4095
4096
4097
4098
4099
4100
4101
4102
4103
4104
4105
4078
4090
4091
4092
4093
4094
4095
4096
4097
4098
4099
4100
4101
4102
4103
4104
4105
4079
4090
4091
4092
4093
4094
4095
4096
4097
4098
4099
4100
4101
4102
4103
4104
4105
4080
4090
4091
4092
4093
4094
4095
4096
4097
4098
4099
4100
4101
4102
4103
4104
4105
4081
4090
4091
4092
4093
4094
4095
4096
4097
4098
4099
4100
4101
4102
4103
4104
4105
4082
4090
4091
4092
4093
4094
4095
4096
4097
4098
4099
4100
4101
4102
4103
4104
4105
4083
4090
4091
4092
4093
4094
4095
4096
4097
4098
4099
4100
4101
4102
4103
4104
4105
4084
4090
4091
4092
4093
4094
4095
4096
4097
4098
4099
4100
4101
4102
4103
4104
4105
4085
4090
4091
4092
4093
4094
4095
4096
4097
4098
4099
4100
4101
4102
4103
4104
4105
4086
4090
4091


4191
4192
4193
4194
4195
4196
4197
4198
4199
4200
4174
4185
4186
4187
4188
4189
4190
4191
4192
4193
4194
4195
4196
4197
4198
4199
4200
4175
4185
4186
4187
4188
4189
4190
4191
4192
4193
4194
4195
4196
4197
4198
4199
4200
4176
4185
4186
4187
4188
4189
4190
4191
4192
4193
4194
4195
4196
4197
4198
4199
4200
4177
4185
4186
4187
4188
4189
4190
4191
4192
4193
4194
4195
4196
4197
4198
4199
4200
4178
4185
4186
4187
4188
4189
4190
4191
4192
4193
4194
4195
4196
4197
4198
4199
4200
4179
4185
4186
4187
4188
4189
4190
4191
4192
4193
4194
4195
4196
4197
4198
4199
4200
4180
4185
4186
4187
4188
4189
4190
4191
4192
4193
4194
4195
4196
4197
4198
4199
4200
4181
4185
4186
4187
4188
4189
4190
4191
4192
4193
4194
4195
4196
4197
4198
4199
4200
4182
4185
4186
4187
4188
4189
4190
4191
4192
4193
4194
4195
4196
4197
4198
4199
4200
4183
4185
4186
4187
4188
4189
4190
4191
4192
4193
4194
4195
4196
4197
4198
4199
4200
4185
4186
4187
4188
4189
4190
4191
4192
4193
4194
4195
4196
4197
4198
4199
4200
4185
4202
4203
4204


4339
4340
4341
4342
can only convert an array of size 1 to a Python scalar
4324
4330
4331
4332
4333
4334
4335
4336
4337
4338
4339
4340
4341
4342
4325
4330
4331
4332
4333
4334
4335
4336
4337
4338
4339
4340
4341
4342
can only convert an array of size 1 to a Python scalar
4326
4330
4331
4332
4333
4334
4335
4336
4337
4338
4339
4340
4341
4342
4327
4330
4331
4332
4333
4334
4335
4336
4337
4338
4339
4340
4341
4342
4328
4330
4331
4332
4333
4334
4335
4336
4337
4338
4339
4340
4341
4342
4330
4331
4332
4333
4334
4335
4336
4337
4338
4339
4340
4341
4342
4330
4344
4345
4346
4347
4348
4349
4350
4351
4352
4353
4354
4355
4356
4357
4358
4359
4331
4344
4345
4346
4347
4348
4349
4350
4351
4352
4353
4354
4355
4356
4357
4358
4359
4332
4344
4345
4346
4347
4348
4349
4350
4351
4352
4353
4354
4355
4356
4357
4358
4359
4333
4344
4345
4346
4347
4348
4349
4350
4351
4352
4353
4354
4355
4356
4357
4358
4359
4334
4344
4345
4346
4347
4348
4349
4350
4351
4352
4353
4354
4355
4356
4357
4358
4359
4335
4344
4345
4346
4347
4348


4465
4466
4467
4468
4469
4470
4471
4472
4473
4474
4475
4476
4477
4465
4479
4480
4481
4482
4483
4484
4485
4486
4487
4488
4489
4490
4491
4492
4466
4479
4480
4481
4482
4483
4484
4485
4486
4487
4488
4489
4490
4491
4492
4467
4479
4480
4481
4482
4483
4484
4485
4486
4487
4488
4489
4490
4491
4492
4468
4479
4480
4481
4482
4483
4484
4485
4486
4487
4488
4489
4490
4491
4492
4469
4479
4480
4481
4482
4483
4484
4485
4486
4487
4488
4489
4490
4491
4492
4470
4479
4480
4481
4482
4483
4484
4485
4486
4487
4488
4489
4490
4491
4492
4471
4479
4480
4481
4482
4483
4484
4485
4486
4487
4488
4489
4490
4491
4492
4472
4479
4480
4481
4482
4483
4484
4485
4486
4487
4488
4489
4490
4491
4492
4473
4479
4480
4481
4482
4483
4484
4485
4486
4487
4488
4489
4490
4491
4492
4474
4479
4480
4481
4482
4483
4484
4485
4486
4487
4488
4489
4490
4491
4492
4475
4479
4480
4481
4482
4483
4484
4485
4486
4487
4488
4489
4490
4491
4492
4476
4479
4480
4481
4482
4483
4484
4485
4486
4487
4488
4489
4490
4491
4492
4477
4479
4480
4481
4482
4483
4484


can only convert an array of size 1 to a Python scalar
4604
4615
4616
4617
4618
4619
4620
4621
4622
4623
4624
4625
4626
4627
4628
4629
4605
4615
4616
4617
4618
4619
4620
4621
4622
4623
4624
4625
4626
4627
4628
4629
4606
4615
4616
4617
4618
4619
4620
4621
4622
4623
4624
4625
4626
4627
4628
4629
4607
4615
4616
4617
4618
4619
4620
4621
4622
4623
4624
4625
4626
4627
4628
4629
4608
4615
4616
4617
4618
4619
4620
4621
4622
4623
4624
4625
4626
4627
4628
4629
4609
4615
4616
4617
4618
4619
4620
4621
4622
4623
4624
4625
4626
4627
4628
4629
4610
4615
4616
4617
4618
4619
4620
4621
4622
4623
4624
4625
4626
4627
4628
4629
4611
4615
4616
4617
4618
4619
4620
4621
4622
4623
4624
4625
4626
4627
4628
4629
4612
4615
4616
4617
4618
4619
4620
4621
4622
4623
4624
4625
4626
4627
4628
4629
4613
4615
4616
4617
4618
4619
4620
4621
4622
4623
4624
4625
4626
4627
4628
4629
4615
4616
4617
4618
4619
4620
4621
4622
4623
4624
4625
4626
4627
4628
4629
4615
4631
4632
4633
4634
4635
4636
4637
4638
4639
4640
4641
4642
4643


In [16]:
bert_threshold_classification.to_csv("threshold_classification.csv", index=False)

In [93]:
find_next_batch(1641, data)
# data[1630:1650]

1642
1643
1644
1645
1646
1647
1648


,index,Country,Date,Cases,Deaths,Summary,Bordering Countries,State
1641,1641,6,2012-04-09,5,0.0,Benin: Two (2) districts are in epidemic with ...,"[19, 18, 11]",0
1642,1643,12,2012-04-09,25,0.0,Benin: Two (2) districts are in epidemic with ...,"[14, 1, 5, 8, 13, 17]",2
1643,1644,14,2012-04-09,8,2.0,Benin: Two (2) districts are in epidemic with ...,"[0, 15, 1, 12, 22]",0
1644,1645,15,2012-04-09,34,4.0,Benin: Two (2) districts are in epidemic with ...,"[0, 3, 14, 22]",1
1645,1646,17,2012-04-09,79,0.0,Benin: Two (2) districts are in epidemic with ...,"[10, 8, 9, 12, 13]",2
1646,1647,19,2012-04-09,28,0.0,Benin: Two (2) districts are in epidemic with ...,"[6, 18, 4, 22]",2
1647,1648,22,2012-04-09,396,19.0,Benin: Two (2) districts are in epidemic with ...,"[19, 4, 3, 14, 15]",2
1648,1649,23,2012-04-09,19,0.0,Benin: Two (2) districts are in epidemic with ...,"[7, 1, 0, 8]",2
